# Data Import and Cleaning

### Import Libaries

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import datetime as dt
from datetime import datetime
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf
from google.colab import files
from google.colab import drive
# drive.mount('/content/drive')

%matplotlib inline

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
df2016 = pd.read_csv("../content/drive/My Drive/Capstone/data/res_statistics2016.txt", sep='\t')
df2017 = pd.read_csv("../content/drive/My Drive/Capstone/data/res_statistics2017.txt", sep='\t')
df2018 = pd.read_csv("../content/drive/My Drive/Capstone/data/res_statistics2018.txt", sep='\t')
df2019 = pd.read_csv("../content/drive/My Drive/Capstone/data/res_statistics2019.txt", sep='\t')
df2020 = pd.read_csv("../content/drive/My Drive/Capstone/data/res_statistics2020.txt", sep='\t')

Concatinating all years into one dataframe:

In [ ]:
df = pd.concat([df2016, df2017, df2018, df2019, df2020])

Let's clear all rows with erroneous entries via their indicies:

In [ ]:
df['RESORT'].unique()

array(['Hotel1', 'S_DEF_ROOM_REPORT', '20069', '38247', '37921', '36905',
       '10423'], dtype=object)

In [ ]:
df = df.drop(df.loc[df['RESORT']=='S_DEF_ROOM_REPORT'].index)

In [ ]:
df = df.drop(df.loc[df['RESORT']=='20069'].index)

In [ ]:
df = df.drop(df.loc[df['RESORT']=='38247'].index)

In [ ]:
df = df.drop(df.loc[df['RESORT']=='37921'].index)

In [ ]:
df = df.drop(df.loc[df['RESORT']=='36905'].index)

In [ ]:
df = df.drop(df.loc[df['RESORT']=='10423'].index)

In [ ]:
df['BUSINESS_DATE'] = pd.to_datetime(df['BUSINESS_DATE'], format='%d-%b-%y')

In [ ]:
df['CHAR_BUSINESS_DATE'] = pd.to_datetime(df['CHAR_BUSINESS_DATE'], format='%m/%d/%y')

Converting some numbers stored as objects to floats:

In [ ]:
to_float = ['S_REV_MKT', 'S_DEF_ROOMS_MKT',
    'S_FB_REV_MKT', 'S_MISC_REV_MKT', 'MKT_RESORT_ROOM', 'GUEST_MKT',
    'TOT_ARR_MKT']
df[to_float] = df[to_float].astype(float)

Formula to retrieve duplicate columns:

In [ ]:
# from https://www.geeksforgeeks.org/how-to-find-drop-duplicate-columns-in-a-pandas-dataframe/
def getDuplicateColumns(df):

    duplicateColumnNames = set()
    for x in range(df.shape[1]):
        col = df.iloc[:, x]

        for y in range(x + 1, df.shape[1]):
            otherCol = df.iloc[:, y]
            if col.equals(otherCol):
                duplicateColumnNames.add(df.columns.values[y])

    return list(duplicateColumnNames)

In [ ]:
getDuplicateColumns(df)

['ARRIVAL_MKT',
 'RESORT_ROOM',
 'CHAR_BUSINESS_DATE',
 'CF_MASTER_SEQ',
 'ARR_TODAY',
 'GROUP_NAME']

Identifying columns with all zero values:

In [ ]:
(df == 0).all()

RESORT                False
BUSINESS_DATE         False
CHAR_BUSINESS_DATE    False
MARKET_CODE           False
CF_MASTER_SEQ         False
GROUP_NAME            False
S_REV_MKT             False
S_DEF_ROOMS_MKT       False
S_FB_REV_MKT          False
S_MISC_REV_MKT        False
MKT_RESORT_ROOM       False
GUEST_MKT             False
TOT_ARR_MKT           False
PER_OCC_MKT           False
PER_DOUBLE_MKT        False
DOUBLE_OCC_MKT        False
SINGLE_OCC_MKT        False
GUEST_DAY             False
ARRIVAL_DAY            True
ARRIVAL_MKT            True
TOT_ARR_DAY           False
PER_OCC_DAY           False
PER_DOUBLE_OCC_DAY    False
SINGLE_OCC_DAY        False
S_MISC_REV_DAY        False
DAY_RESORT_ROOM       False
S_FB_REV_DAY          False
DOUBLE_OCC_DAY        False
S_REV_DAY             False
S_DEF_ROOMS_DAY       False
ARR_TODAY              True
NO_DEFINITE_ROOMS     False
IN_GUEST              False
OCC_SINGLE            False
DOUBLE_OCC            False
REVENUE             

Dropping all columns from the 3 steps above:

In [ ]:
df = df.drop(columns=['RESORT', 'ARRIVAL_DAY', 'CHAR_BUSINESS_DATE',
                      'ARRIVAL_MKT',
                      'CF_MASTER_SEQ',
                      'ARR_TODAY',
                      'RESORT_ROOM',
                      'GROUP_NAME'], axis=1)

In [ ]:
df.shape # came down from 43 to 35 columns

(23004, 35)

In [ ]:
# disabling max columns to show in head() to review all columns
pd.options.display.max_columns = None
df.tail(2) # reviewing all columns

,BUSINESS_DATE,MARKET_CODE,S_REV_MKT,S_DEF_ROOMS_MKT,S_FB_REV_MKT,S_MISC_REV_MKT,MKT_RESORT_ROOM,GUEST_MKT,TOT_ARR_MKT,PER_OCC_MKT,PER_DOUBLE_MKT,DOUBLE_OCC_MKT,SINGLE_OCC_MKT,GUEST_DAY,TOT_ARR_DAY,PER_OCC_DAY,PER_DOUBLE_OCC_DAY,SINGLE_OCC_DAY,S_MISC_REV_DAY,DAY_RESORT_ROOM,S_FB_REV_DAY,DOUBLE_OCC_DAY,S_REV_DAY,S_DEF_ROOMS_DAY,NO_DEFINITE_ROOMS,IN_GUEST,OCC_SINGLE,DOUBLE_OCC,REVENUE,FB_REV,OTHER_REV,TOTAL_REVENUE,PER_OCC,GET_ARR,MULTI_OCC_PER
4390,2020-12-31,RETAIL,783339.36,3912.0,750.0,235618.06,42456.0,5879.0,200.240123,9.214245,4.159601,1766.0,2146.0,43.0,131.277857,24.137931,12.931035,13.0,1341.2,116.0,0.0,15.0,3675.78,28.0,12.0,21.0,4.0,8.0,1874.46,0.0,607.98,2482.44,10.344828,156.205,6.896552
4391,2020-12-31,WHOL,4355.45,26.0,0.0,1049.34,42456.0,37.0,167.517308,0.061240,0.025909,11.0,15.0,43.0,131.277857,24.137931,12.931035,13.0,1341.2,116.0,0.0,15.0,3675.78,28.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.00,0.000000,0.000,0.000000


Renaming columns to lowercase per pythonic conventions

In [ ]:
df.columns = [_.lower() for _ in df.columns]

Based on domaain knowledege, some market segments are not utilized. Let's identify those and remove all rows with them. This will also help trim down the features once this column gets dummified.

In [ ]:
df.market_code.unique()

array(['AR', 'COMP', 'CONT', 'DISC', 'GASSOC', 'GCOR', 'GSMERF', 'HOUSE',
       'NEG', 'PSBY', 'QUAL', 'RETAIL', 'WHOL', 'GWHOL'], dtype=object)

In [ ]:
'''
AR = Accounts Receivable
COMP = Complimentary
CONT = Contract
DISC = Discounts
GASSOC = Group - Association
GCOR = Group - Corporate
GSMERF = Group - SMERF
HOUSE = House Use
NEG = Negotiated
PSBY = Passerby
QUAL = Qualified Discount
RETAIL = Retail/BAR
WHOL = Wholesale
GWHOL = Group - Wholesale
'''

'\nAR = Accounts Receivable\nCOMP = Complimentary\nCONT = Contract\nDISC = Discounts\nGASSOC = Group - Association\nGCOR = Group - Corporate\nGSMERF = Group - SMERF\nHOUSE = House Use\nNEG = Negotiated\nPSBY = Passerby\nQUAL = Qualified Discount\nRETAIL = Retail/BAR\nWHOL = Wholesale\nGWHOL = Group - Wholesale\n'

Let's check if there are columns with zero or close to zero values: 

In [ ]:
df[['no_definite_rooms','revenue']].groupby(df['market_code']).sum()

,no_definite_rooms,revenue
market_code,,
AR,0.0,551.47
COMP,1071.0,-55.96
CONT,4.0,747.04
DISC,36725.0,8710289.24
GASSOC,4848.0,1681113.21
GCOR,19397.0,7217624.67
GSMERF,745.0,197715.50
GWHOL,110.0,33280.00
HOUSE,121.0,-3848.84


"AR", "CONT" and "PSBY" can be dropped as they do not contain significant values and, as confirmed by domain experts, are not utilized anymore.

In [ ]:
index_names = df[ (df['market_code'] == 'AR') | (df['market_code'] == 'CONT') | (df['market_code'] == 'PSBY')].index

In [ ]:
df = df.drop(index_names)

The subject hotel opened in July 2017, yet data was pulled form January 1, 2017. This data will need to be dropped as it will affect our models. 

In [ ]:
df = df[~(df['business_date'] < '2017-07-01')]

,business_date,market_code,s_rev_mkt,s_def_rooms_mkt,s_fb_rev_mkt,s_misc_rev_mkt,mkt_resort_room,guest_mkt,tot_arr_mkt,per_occ_mkt,per_double_mkt,double_occ_mkt,single_occ_mkt,guest_day,tot_arr_day,per_occ_day,per_double_occ_day,single_occ_day,s_misc_rev_day,day_resort_room,s_fb_rev_day,double_occ_day,s_rev_day,s_def_rooms_day,no_definite_rooms,in_guest,occ_single,double_occ,revenue,fb_rev,other_rev,total_revenue,per_occ,get_arr,multi_occ_per
2356,2017-07-01,GASSOC,272910.00,759.0,41187.72,6420.04,42340.0,790.0,359.565217,1.792631,0.073217,31.0,728.0,222.0,211.321071,96.551724,72.413793,28.0,4003.38,116.0,0.0,84.0,23667.96,112.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
2357,2017-07-01,GCOR,1530162.34,4777.0,620649.64,130252.92,42340.0,5079.0,320.318681,11.282475,0.727444,308.0,4469.0,222.0,211.321071,96.551724,72.413793,28.0,4003.38,116.0,0.0,84.0,23667.96,112.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
2358,2017-07-01,GSMERF,45759.10,170.0,0.00,5278.07,42340.0,202.0,269.171176,0.401512,0.073217,31.0,139.0,222.0,211.321071,96.551724,72.413793,28.0,4003.38,116.0,0.0,84.0,23667.96,112.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
2359,2017-07-01,GWHOL,10619.00,41.0,0.00,1554.00,42340.0,41.0,259.000000,0.096835,0.000000,0.0,41.0,222.0,211.321071,96.551724,72.413793,28.0,4003.38,116.0,0.0,84.0,23667.96,112.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
2361,2017-07-01,NEG,3133010.95,10003.0,17011.86,100826.38,42340.0,10358.0,313.207133,23.625413,0.772319,327.0,9676.0,222.0,211.321071,96.551724,72.413793,28.0,4003.38,116.0,0.0,84.0,23667.96,112.0,6.0,6.0,6.0,0.0,1330.2,0.0,186.0,1516.2,5.172414,221.7,0.0


In [ ]:
df.to_csv('../content/drive/My Drive/Capstone/data/cleaned_data.csv')